Le cinque funzioni permettono di:
- leggere il contenuto del corpus e inserirlo in una variabile (`contents`), su cui lavoro per non accedere direttamente al testo
- fare il sentence splitting del testo, le cui frasi finiscono nella variabile `sentences`, con la funzione di nltk `nltk.tokenize.sent_tokenize`
- a partire dalle frasi (estratte nella seconda funzione), questa funzione seleziona i token per ogni frase con una iterazione e li inserisce con l'operazione `.append` in un nuovo array (`tokens_in_sentences`) dove ogni frase è una lista di liste (ognuna al proprio interno ha i suoi token), mentre con l'operazione `.extend` inserisco tutti i token in una unica lista (`all_tokens`)
- restituire, a partire dai token del corpus, il corpus annotato morfo-sintatticamente tramite la funzione di nltk `nltk.tag.pos_tag()`. Ottengo una lista contentente coppie [token, Pos]. La PoS, se non è specificato un tagset, viene calcolata sul Penn Treebank.
- restituire una lista contente solo le POS del testo a partire dal corpus annotato. Con un ciclo `for` estrae la PoS da ogni coppia parola-PoS e la inserisce in una nuova lista (`only_POS`).

In [1]:
import nltk
import math

def read_file_content(file):
    with open(file, 'r', encoding="utf8") as infile:
        contents = infile.read()
        return contents

def get_sentences(text):
    sentences = nltk.tokenize.sent_tokenize(text)
    return sentences

def get_tokens(text_sentences):
    all_tokens = []
    tokens_in_sentences = []
    for sentence in text_sentences:
        tokens = nltk.tokenize.word_tokenize(sentence)
        all_tokens.extend(tokens)
        tokens_in_sentences.append(tokens)
    return all_tokens, tokens_in_sentences

def annotate(tokens):
     tokens_POS = nltk.tag.pos_tag(tokens) 
     return tokens_POS

def get_POS_only(tokens_POS):
    only_POS = []
    for token, POS in tokens_POS:
        only_POS.append(POS)
    return only_POS

1a.**Calcolo della sequenza ordinata in ordine descrescente dei primi 10 PoS, bigrammi di Pos e trigrammi di PoS più frequenti**.
La funzione `freq_ngrams_POS` restituisce gli n-grammi di PoS più frequenti. In input ha la lista delle PoS del testo (`POS_list`), il grado degli n-grammi(`grams`) da estrarre e il numero preciso di n-grammi più frequenti, ordinati in ordine decrescente, che voglio (`i`). La prima azione che compie è dividere la lista di PoS in ngrammi secondo il grado che gli ho fornito in input usando la funzione `nltk.ngrams(lista_di_PoS, n = grado)`. A questo punto il valore (la lista di n-grammi) viene inserito in una variabile (`ngrams`), di cui verrà calcolata la distribuzione di frequenza con `nltk.FreqDist()`. Infine della distribuzione di frequenza (ovvero un dizionario [n-gramma: frequenza]) sceglieremo i primi `i` valori in ordine decrescente con `.most_common(i)`.

1b.**Calcolo dei 20 sostantivi, avverbi e aggettivi più frequenti**. La funzione prende in input il corpus annotato, un array contentente le POS che mi servono e un indice `i` che mi specifica quanti valori mi deve restituire. Le opzioni sono tre, ognuna per la POS di cui ho bisogno: se la POS che voglio trovare è un aggettivo, inserirò nella variabile `POS` un array con i tag corrispondenti agli aggettivi e accederò alle operazioni per quella specifica POS. Faccio così anche per nomi e avverbi. Le operazioni per ogni POS sono: 1.la funzione scorre il corpus annotato e prenderà i token che hanno una POS che inizia come la POS che mi serve (JJ pe l'aggettivo, NN per il sostantivo e RB per l'avverbio), 2.aggiungerà il token ad una lista `l` contentente i token del testo con quella specifica POS, di cui calcolo la distribuzione di frequenza(`nltk.FreqDist(l)`), e 3. resituirò gli `n` token più frequenti nella distribuzione(`l_freq.most_common(n)`). Inoltre, quando filtro i token di cui ho bisogno nel corpus annotato ho aggiunto che devono essere anche più lunghi di 1, altrimenti mi inserisce anche gli apostrofi.


In [2]:
#1a
def freq_ngrams_POS(POS_list, grams, i):
    ngrams = nltk.ngrams(POS_list, n = grams)
    freq_ngrams_POS = nltk.FreqDist(ngrams)
    return freq_ngrams_POS.most_common(i)

#1b
def freq_POS(tokens_POS, POS, n):
    l= []
    if ('JJ' in POS and 'JJR' in POS and 'JJS' in POS):
        for token, token_POS in tokens_POS:
            if(token_POS.startswith('JJ')and len(token)>1):
                l.append(token)
        l_freq = nltk.FreqDist(l)
        return l_freq.most_common(n)
    if ('RB' in POS and 'RBR' in POS and 'RBS' in POS):
        for token, token_POS in tokens_POS:
            if(token_POS.startswith('RB')and len(token)>1):
                l.append(token)
        l_freq = nltk.FreqDist(l)
        return l_freq.most_common(n)
    if ('NN'in POS and 'NNS' in POS and 'NNP' in POS and 'NNPS' in POS):
        for token, token_POS in tokens_POS:
            if(token_POS.startswith('NN') and len(token)>1):
                l.append(token)
        l_freq = nltk.FreqDist(l)
        return l_freq.most_common(n)


2. **Estratti i bigrammi composti da Aggettivo e Sostantivo mostare:**
- a. I 20 più frequenti, con relativa frequenza
- b. I 20 con probabilità condizionata massima, e relativo valore di probabilità
- c. I 20 con forza associativa (Pointwise Mutual Information, PMI) massima, e
relativa PMI

La prima funzione (`get_adj_noun_bigrams`) estrae i bigrammi aggettivo-ostantivo a partire dal corpus annotato. Attraverso `nltk.bigrams` divido il corpus annotato per bigrammi, così da ottenre coppie di token con le rispettive PoS. A questo punto tra questi token prendo solo quelli che hanno come PoS del primo token un aggettivo (`.startswith('JJ'))` e del secondo un sostantivo (`element_2[1].startswith('NN'))`. Ho aggiunto anche che i token siano più lunghi di così da evitare apostrofi o altri segni. Infine, selezionati i bigrammi di token aggettivo-ostantivo creo un bigramma dove inserisco solo i token del bigramma (`element_1[0], element_2[0]`) e li inserisco in una lista.

- a. La funzione `get_freq_adj_noun_bigrams` prende in input i bigrammi aggettivo-sostantivo e un valore nuemrico `i`. Calcola la distribuzione di frequenza dei bigrammi (`freq_adj_noun_bigrams`) e restituisce gli i bigrammi più grandi (`freq_adj_noun_bigrams.most_common(i)`)

- b. La funzione `get_cond_prob_adj_noun`prende in input i bigrammi aggettivo-sostantivo, i token del corpus e un valore nuemrico `i`. Calcola la distribuzione di frequenza di tutti i bigrammi del testo (`freq_bigrams`) e dei token (`freq_tokens`). Applicando un ciclo `for`, per ogni bigramma aggettivo-sostantivo calcola la probabilità condizionata (frequenza del bigramma nel corpus/frequenza del primo elemento del bigramma) a partire dalle distribuzioni di frequenza calcolate all'inizio. Poi ogni volta crea un bigramma (`bigram`) dove inserisce la coppia aggettivo-sostantivo come primo elemento e la relativa probabilità condizionata come secondo. Aggiunge ogni bigramma in una lista (`prob_cond_all_bigrams`) che va ordinata in modo decrescente (secondo i valori di probabilità) e di cui vanno presi i primi `i` elementi. La funzione `sorted()` ordina la lista in modo crescente di default, con `reverse=True` la ordina in modo decrescente. Per ordinarla a partire dai secondi valori degli elementi (le probabilità), uso `key` all'interno di sorted che richiama una funzione `take_second`, la quale restituisce il secondo valore di un elemento. Infine, dopo aver creato la lista di bigrammi ordinati in modo decrescente a partire dalle probabilità (`ordered_prob_cond_all_bigrams`), la funzione restituisce gli elementi di questa lista dall'inizio fino ad `i` con lo slicing (`ordered_prob_cond_all_bigrams[:i]`).

- c. La funzione `get_PMI_adj_noun`prende in input i bigrammi aggettivo-sostantivo, i token del corpus e un valore nuemrico `i`. Calcola la distribuzione di frequenza di tutti i bigrammi del testo (`freq_bigrams`) e dei token (`freq_tokens`). Applicando un ciclo `for`, per ogni bigramma aggettivo-sostantivo calcola la Pointwise Mutual Information (probabilità del bigramma/probabilità del primo elemento del bigramma per il secondo) a partire dalle distribuzioni di frequenza calcolate all'inizio. Per trovare le probabilità divide infatti le frequenze per la lunghezza del corpus (`len(tokens)`), anche per i bigrammi (numero di bigrammi = numero di token). Poi ogni volta crea un bigramma (`bigram`) dove inserisce la coppia aggettivo-sostantivo come primo elemento e la relativa Pointwise Mutual Information come secondo. Aggiunge ogni bigramma in una lista (`PMI_all_bigrams`) che va ordinata in modo decrescente (secondo i valori di PMI) e di cui vanno presi i primi `i` elementi. La funzione `sorted()` ordina la lista in modo crescente di default, con `reverse=True` la ordina in modo decrescente. Per ordinarla a partire dai secondi valori degli elementi (ovvero la PMI di ognuno), uso `key` all'interno di sorted che richiama una funzione `take_second`, la quale restituisce il secondo valore di un elemento. Infine, dopo aver creato la lista di bigrammi ordinati in modo decrescente a partire dalle PMI (`ordered_PMI_all_bigrams`), la funzione restituisce gli elementi di questa lista dall'inizio fino ad `i` con lo slicing (`ordered_PMI_all_bigrams[:i]`).

In [3]:
def get_adj_noun_bigrams(tokens_POS):
    tokens_POS_bigrams = list(nltk.bigrams(tokens_POS))
    adj_noun_bigrams = []
    for element_1, element_2 in tokens_POS_bigrams:
        if (element_1[1].startswith('JJ')) and (element_2[1].startswith('NN') and len(element_1[0])>1 and len(element_2[0])>1):
            bigram = (element_1[0], element_2[0])
            adj_noun_bigrams.append(bigram)
    return adj_noun_bigrams

#a
def get_freq_adj_noun_bigrams(adj_noun_bigrams, i):
    freq_adj_noun_bigrams = nltk.FreqDist(adj_noun_bigrams)
    return  freq_adj_noun_bigrams.most_common(i)

#b
def get_cond_prob_adj_noun(adj_noun_bigrams, tokens, i):
    freq_bigrams = nltk.FreqDist(nltk.bigrams(tokens))
    freq_tokens = nltk.FreqDist(tokens)
    prob_cond_all_bigrams = []
    for token_1, token_2 in adj_noun_bigrams:
        prob_cond_bigram = freq_bigrams[(token_1, token_2)]/freq_tokens[token_1]
        bigram = ((token_1, token_2), prob_cond_bigram)
        prob_cond_all_bigrams.append(bigram)
    ordered_prob_cond_all_bigrams = sorted(prob_cond_all_bigrams, key=take_second, reverse=True)
    return ordered_prob_cond_all_bigrams[:i]

def take_second(elem):
    return elem[1]

#c
def get_PMI_adj_noun(adj_noun_bigrams, tokens, i):
    freq_tokens = nltk.FreqDist(tokens)
    freq_bigrams = nltk.FreqDist(nltk.bigrams(tokens))
    PMI_all_bigrams = []
    for token_1, token_2 in adj_noun_bigrams:
        PMI = math.log2((freq_bigrams[(token_1, token_2)]/(len(tokens)))/((freq_tokens[token_1]/len(tokens))*((freq_tokens[token_2]/len(tokens)))))
        bigram = ((token_1, token_2), PMI)
        PMI_all_bigrams.append(bigram)
    ordered_PMI_all_bigrams = sorted(PMI_all_bigrams, key=take_second, reverse=True)
    return ordered_PMI_all_bigrams[:i]


3.**Considerate le frasi con una lunghezza compresa tra 10 e 20 token, in cui almeno la metà dei token occorre almeno 2 volte nel corpus, si identifichino:**
- a. La frase con la media della distribuzione di frequenza dei token più alta
- b. La frase con la media della distribuzione di frequenza dei token più bassa
- c. La frase con probabilità più alta secondo un modello di Markov di ordine 2
costruito a partire dal corpus di input

La funzione `get_specific_sentences` prende in input le frasi divise in token (che ho estratto con la funzione `get_tokens` - vedi sopra) e i token. Per ogni frase, se la frase ha lunghezza compresa tra 10 e 20, conto ogni token con frequenza maggiore di 1 attraverso una variabile `i` che sarà poi azzerata passando alla frase successiva. Se questa variabile sarà maggiore-uguale della metà della lunghezza della frase (quindi i token con frequenza maggiore di 1 sono almeno la metà dei token della frase), aggiungerò la frase in una lista (`specific_sentences`). La funzione restituirà questa lista.

- a. La funzione `get_max_sent_freq_dist` prende in input le frasi selezionate con la funzione `get_specific_sentences` e i token. Iterando su ogni frase e poi su ogni token di ogni frase, somma la frequenza di ogni token nella frase (a partire dlla distribuzione di frequenza `freq_tokens`), la divide per la lunghezza della frase e inserisce il valore nella variabile `sent_freq_dist` (che torna 0 pasando alla frase successiva così come la somma delle frequenze dei token `freq_all_tokens`). Se questo valore è maggiore di `max_sent_freq_dist` (che all'inizio sarà 0), la frase diventa `max_sent`, ovvero la frase con la media della distribuzione di frequenza dei token più alta. La funzione restituirà `max_sent`.

- b. La funzione `get_min_sent_freq_dist` prende in input le frasi selezionate con la funzione `get_specific_sentences` e i token. Iterando su ogni frase e poi su ogni token di ogni frase, somma la frequenza di ogni token nella frase (a partire dlla distribuzione di frequenza `freq_tokens`), la divide per la lunghezza della frase e inserisce il valore nella variabile `sent_freq_dist` (che torna 0 pasando alla frase successiva così come la somma delle frequenze dei token `freq_all_tokens`). Se questo valore è minore di `min_sent_freq_dist` (che all'inizio sarà infinto), la frase diventa `min_sent`, ovvero la frase con la media della distribuzione di frequenza dei token più bassa. La funzione restituirà `min_sent`.

- c. La funzione `markov2` calcola la probabilità di una frase a partire dalle distribuzioni in un corpus. Sia la frase che i token del corpus sono forniti in input. Calcolando la probabilità su un modello di Markov di ordine 2, avrò bisogno di dividere i token e la frase in bigrammi e trigrammi (`bigrams`, `trigrams`, `bigrams_sentence`, `trigrams_sentence`). Calcola quindi la distribuzione di frequenza dei token, dei bigrammi e dei trigrammi del testo (`freq_tokens`, `freq_bigrams`, `freq_trigrams`). Viene poi calcolata la probabilità della prima parola della frase(`token_1_prob`) e quella condizionata della seconda (`token_2_prob`). Poi su ogni trigramma della frase viene calcolata la probabilità condizionata (`trigram_prob`), aggiornando il bigramma nella divisione con una variabile `i`. Infine la funzione restituisce il prodotto di `token_1_prob`, `token_2_prob` e `trigram_prob`, ovvero la probabilità della frase.

La funzione `get_max_sent_prob` prende in input le frasi selezionate con la funzione `get_specific_sentences` e i token. Per ogni frase viene calcolata la probabilità con la funzione `markov2`. Se la probabilità è maggiore di `max_sent_prob` (che inizialmente è 0), allora la frase diventa `max_sent`, la frase con probabilità più alta, e la sua probabilità diventa `max_sent_prob`. Dopo aver iterato su ogni frase, la funzione resituirà la `max_sent` tra le frasi date in input, basandosi sulla distribuzione di probabilità del corpus fornito in input.



In [4]:
def get_specific_sentences(divided_sentences, tokens):
    freq_tokens =  nltk.FreqDist(tokens)
    i = 0
    specific_sentences = []
    for sentence in divided_sentences:
        if len(sentence)>10 and len(sentence)<20:
            for token in sentence:
                if freq_tokens[token]>1:
                    i = i + 1
            if i >= (len(sentence)//2):
                specific_sentences.append(sentence)
            i = 0 
    return specific_sentences

#a
def get_max_sent_freq_dist(specific_sentences, tokens):
    freq_tokens =  nltk.FreqDist(tokens)
    freq_all_tokens = 0
    max_sent_freq_dist = 0
    for sentence in specific_sentences:
        for token in sentence:
            freq_all_tokens = freq_all_tokens + freq_tokens[token]
        sent_freq_dist = freq_all_tokens/len(sentence)
        if sent_freq_dist>max_sent_freq_dist:
            max_sent_freq_dist = sent_freq_dist
            max_sent = sentence
        freq_all_tokens = 0
        sent_freq_dist = 0
    return max_sent

#b
def get_min_sent_freq_dist(specific_sentences, tokens):
    freq_tokens =  nltk.FreqDist(tokens)
    freq_all_tokens = 0
    min_sent_freq_dist = float('inf')
    for sentence in specific_sentences:
        for token in sentence:
            freq_all_tokens = freq_all_tokens + freq_tokens[token]
        sent_freq_dist = freq_all_tokens/len(sentence)
        if sent_freq_dist<min_sent_freq_dist:
            min_sent_freq_dist = sent_freq_dist
            min_sent = sentence
        freq_all_tokens = 0
        sent_freq_dist = 0
    return min_sent

#c
def markov2(sentence, tokens):
    freq_tokens =  nltk.FreqDist(tokens)
    bigrams = list(nltk.bigrams(tokens))
    trigrams = list(nltk.ngrams(tokens, n=3))
    freq_bigrams = nltk.FreqDist(bigrams)
    freq_trigrams = nltk.FreqDist(trigrams)
    bigrams_sentence = list(nltk.bigrams(sentence))
    trigrams_sentence = list(nltk.ngrams(sentence, n=3))
    prob = 1.0
    token_1_prob = freq_tokens[sentence[0]]*1.0/len(tokens)*1.0
    token_2_prob = freq_bigrams[bigrams_sentence[0]]*1.0/freq_tokens[sentence[0]]*1.0
    i = 0
    for trigram in trigrams_sentence:
        trigram_prob = freq_trigrams[trigram]*1.0/freq_bigrams[bigrams_sentence[i]]*1.0
        prob = prob*trigram_prob
        i=i+1
    return prob*token_1_prob*token_2_prob

def get_max_sent_prob(specific_sentences, tokens):
    max_sent_prob = 0
    for sentence in specific_sentences:
        if markov2(sentence, tokens)>max_sent_prob:
            max_sent_prob = max_sent_prob
            max_sent = sentence
    return max_sent

4.**Estratte le Entità Nominate del testo, identificare per ciascuna classe di NE i 15
elementi più frequenti, ordinati per frequenza decrescente e con relativa frequenza.**

La funzione `get_NE` prende in input il corpus annotato e restituisce un array di coppie (parola, entità della parola). Dopo aver creato l'albero delle entità con la funzione `nltk.ne_chunk()`, per ogni nodo dell'albero estrae l'entità (`entity_type`) e il token a cui corrisponde (`entity`). Per fare ciò pongo che il nodo debba avere un attributo di tipo label (`hasattr(nodo, 'label')`), ovvero che sia un nodo con una NE, e scorro le foglie del nodo (`nodo.leaves()`) per ottenere gli elementi di quella NE. Questi elementi sono però nella forma token-POS, quindi prendo solo il token come `entity` .Infine appende la coppia (`entity`, `entity_type`) all'array NE, che viene restituito come output.

La funzione `freq_NE` prende in input le coppie (token, NE) estratte dalla funzione `get_NE`, una classe specifica di NE e un valore numerico `i`. Iterando su ogni elemento della lista di coppie (token, NE), se il secondo elemento (la NE) è uguale alla NE che viene fornita in input, aggiungo il primo elemento (il token) in una lista `l`. Infine viene calcolata la distribuzione di frequenza della lista `l` e ne vengono restituiti gli `i` valori più grandi.
La funzione verrà quindi chiamata tre volte, una per ogni `NE_type` diverso (GPE, PERSON e ORGANIZATION).


In [5]:
def get_NE(tagged_corpus):
    NE_tree = nltk.ne_chunk(tagged_corpus)
    NE = []
    for nodo in NE_tree:
        if hasattr(nodo, 'label'):
            entity_type = nodo.label()
            for token, POS in nodo.leaves():
                entity = token
                NE.append((entity, entity_type))
    return NE

def freq_NE(NE, NE_type, i):
    l = []
    for element in NE:
         if element[1] == NE_type:
            l.append(element[0])
    l_freq = nltk.FreqDist(l)
    return l_freq.most_common(i)
            


La funzione `main` racchiude tutte le funzioni create in precedenza, applicandole al testo fornito in input (`Heart_of_darkness.txt`) e stampando il risultato.

In [6]:
def main(file):


    #eggo il contenuto del file e lo inserisco in una variabile
    corpus = read_file_content(file)


    #faccio il sentence splitting del testo e lo inserisco in una variabile
    sentences = get_sentences(corpus)


    #calcolo i token e i token per frase del corpus 
    tokens, tokens_in_sentence = get_tokens(sentences)


    #faccio l'annotazione e ed estraggo la lista di PoS 
    tagged_corpus = annotate(tokens)
    POS_list = get_POS_only(tagged_corpus)


    #calcolo i primi 10 POS, i primi 10 bigrammi di POS e i primi 10 trigrammi di POS
    ten_POS = freq_ngrams_POS(POS_list, 1, 10)
    ten_bigr_POS = freq_ngrams_POS(POS_list, 2, 10)
    ten_trigr_POS = freq_ngrams_POS(POS_list, 3, 10)

    print(f" - Le 10 PoS più frequenti sono:")
    print(f" ")
    print(f" {ten_POS}")
    print(f" ")
    print(f" - I 10 bigrammi di PoS più frequenti sono:")
    print(f" ")
    print(f" {ten_bigr_POS}")
    print(f" ")
    print(f" - I 10 trigrammi di PoS più frequenti sono:")
    print(f" ")
    print(f" {ten_trigr_POS}")
    print(f" ")


    #Calcolo dei 20 sostantivi più frequenti
    POS_nouns = ['NN', 'NNS', 'NNP', 'NNPS']
    freq_POS_nouns = freq_POS(tagged_corpus, POS_nouns, 20)
    
    print(f" - I 20 sostantivi più frequenti sono:")
    print(f" ")
    print(f" {freq_POS_nouns}")
    print(f" ")


    #Calcolo dei 20 aggettivi più frequenti
    POS_adj = ['JJ', 'JJR', 'JJS']
    freq_POS_adj = freq_POS(tagged_corpus, POS_adj, 20)
    
    print(f" - I 20 aggettivi più frequenti sono:")
    print(f" ")
    print(f" {freq_POS_adj}")
    print(f" ")


    #Calcolo dei 20 avverbi più frequenti
    POS_adv = ['RB', 'RBR', 'RBS']
    freq_POS_adv = freq_POS(tagged_corpus, POS_adv, 20)
    
    print(f" - I 20 avverbi più frequenti sono:")
    print(f" ")
    print(f" {freq_POS_adv}")
    print(f" ")
    

    #Estrazione dei bogrammi aggettivo-sostantivo
    adj_noun_bigrams = get_adj_noun_bigrams(tagged_corpus)

    #Calcolo dei 20 bigrammi più frequenti
    adj_noun_bigrams_freq = get_freq_adj_noun_bigrams(adj_noun_bigrams, 20)

    print(f' - I 20 bigrammi aggettivo-sostantivo più frequenti sono:')
    print(f" ")
    print(f" {adj_noun_bigrams_freq}")
    print(f" ")
    

    #Calcolo dei 20 bigrammi con probabilità condizionata massima
    cond_prob_adj_noun = get_cond_prob_adj_noun(adj_noun_bigrams, tokens, 20)

    print(f' - I 20 bigrammi aggettivo-sostantivo con probabilità condizionata massima sono:')
    print(f" ")
    print(f" {cond_prob_adj_noun}")
    print(f" ")


    #Calcolo dei 20 bigrammi con PMI massima:
    PMI_adj_noun = get_PMI_adj_noun(adj_noun_bigrams, tokens, 20)

    print(f' - I 20 bigrammi aggettivo-sostantivo con forza associativa massima sono:')
    print(f" ")
    print(f" {PMI_adj_noun}")
    print(f" ")


    #Selezione delle frasi con una lunghezza compresa tra 10 e 20 token, in cui almeno la metà dei token occorre almeno 2 volte nel corpus
    text_specific_sentences = get_specific_sentences(tokens_in_sentence, tokens)

    #Calcolo della frase con la media della distribuzione di frequenza dei token più alta
    max_freq_sent = get_max_sent_freq_dist(text_specific_sentences, tokens)

    print(f' - La frase con la media della distribuzione di frequenza dei token più alta è:')
    print(f" ")
    print(f" {max_freq_sent}")
    print(f" ")


    #Calcolo della frase con la media della distribuzione di frequenza dei token più bassa
    min_freq_sent = get_min_sent_freq_dist(text_specific_sentences, tokens)

    print(f' - La frase con la media della distribuzione di frequenza dei token più bassa è:')
    print(f" ")
    print(f" {min_freq_sent}")
    print(f" ")


    #Calcolo della frase con probabilità più alta secondo un modello di Markov di ordine 2 costruito a partire dal corpus di input
    max_sent_prob = get_max_sent_prob(text_specific_sentences, tokens)

    print(f' - La frase con con probabilità più alta secondo un modello di Markov di ordine 2 è:')
    print(f" ")
    print(f" {max_sent_prob}")
    print(f" ")
    

    #Estrazione delle NE del corpus
    text_NE = get_NE(tagged_corpus)

    #Calcolo dei 15 elementi più frequenti della classe PERSON
    text_freq_PERSON = freq_NE(text_NE, 'PERSON', 15)

    print(f' - I 15 elementi della classe PERSON più frequenti sono:')
    print(f" ")
    print(f" {text_freq_PERSON}")
    print(f" ")

    #Calcolo dei 15 elementi più frequenti della classe GPE (Geo-Political Entity)
    text_freq_GPE = freq_NE(text_NE, 'GPE', 15)

    print(f' - I 15 elementi della classe GPE più frequenti sono:')
    print(f" ")
    print(f" {text_freq_GPE}")
    print(f" ")

    #Calcolo dei 15 elementi più frequenti della classe ORGANIZATION
    text_freq_ORG = freq_NE(text_NE, 'ORGANIZATION', 15)

    print(f' - I 15 elementi della classe ORGANIZATION più frequenti sono:')
    print(f" ")
    print(f" {text_freq_ORG}")
    print(f" ")


file = "Heart_of_darkness.txt"
main(file)

 - Le 10 PoS più frequenti sono:
 
 [(('NN',), 6534), (('IN',), 5104), (('DT',), 4406), (('PRP',), 3546), (('VBD',), 3218), (('JJ',), 3145), ((',',), 2849), (('.',), 2293), (('RB',), 2275), (('NNS',), 1526)]
 
 - I 10 bigrammi di PoS più frequenti sono:
 
 [(('DT', 'NN'), 2468), (('IN', 'DT'), 2044), (('NN', 'IN'), 1709), (('PRP', 'VBD'), 1707), (('JJ', 'NN'), 1475), (('NN', ','), 1102), (('DT', 'JJ'), 1102), (('NN', '.'), 950), (('.', 'PRP'), 764), (('IN', 'PRP'), 627)]
 
 - I 10 trigrammi di PoS più frequenti sono:
 
 [(('IN', 'DT', 'NN'), 1221), (('DT', 'NN', 'IN'), 817), (('DT', 'JJ', 'NN'), 783), (('NN', 'IN', 'DT'), 679), (('.', 'PRP', 'VBD'), 538), (('IN', 'DT', 'JJ'), 492), (('JJ', 'NN', 'IN'), 383), (('DT', 'NN', ','), 378), (('DT', 'NN', '.'), 325), (('NN', '.', 'PRP'), 319)]
 
 - I 20 sostantivi più frequenti sono:
 
 [('Kurtz', 112), ('man', 105), ('time', 74), ('river', 53), ('Mr.', 49), ('men', 45), ('head', 45), ('eyes', 44), ('manager', 42), ('station', 41), ('something